In [1]:
import gevent.monkey
gevent.monkey.patch_all(thread=False)

True

C:\Program Files\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
import logging
from steam.client import SteamClient
from csgo.client import CSGOClient
from steam.enums import EResult
from csgo import sharecode
import bz2
from urllib import request
from google.protobuf.json_format import MessageToJson
import json
import bz2
from urllib import request
import queue


from http.server import BaseHTTPRequestHandler, HTTPServer
from io import BytesIO
from urllib.parse import parse_qs

C:\Program Files\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [3]:
class GoClient(CSGOClient):
    decoded = queue.Queue()

In [4]:
logging.basicConfig(format="%(asctime)s | %(message)s", level=logging.INFO)
LOG = logging.getLogger()

client = SteamClient()
client.set_credential_location(".")

goclient = GoClient(client)

@goclient.on("ready")
def handle_ready():
    goclient.request_full_match_info(**goclient.decoded.get())
    goclient.decoded.task_done()

@goclient.on("full_match_info")
def handle_full_match_info(msg):
    goclient.exit()
#     goclient.request_recent_user_games(goclient.account_id)
#     print(str(goclient.wait_msg("recent_user_games"))[0:100])
    msg = MessageToJson(msg)
    links = find_values(msg)
    if len(links) == 0:
        return
    demo_url = links[0]
    demo_name = demo_url.split('/')[-1].split('.')[0]
    path_to_data = "D:\\Dev\\demoparse-data\\"
    path_to_bz2 = path_to_data + "bz2\\" + demo_url.split('/')[-1]
    path_to_dem = path_to_data + "dem\\" + demo_name + '.dem'
    with bz2.open(path_to_bz2, 'rb') as f:
        content = f.read()
    with open(path_to_dem, 'wb') as of:
        of.write(content)
    path_to_demoparser_executable = "c:\\Users\\admin\\go\\src\\csgo-parser-mongodb\\bin\\demoparse-mongo.exe"
    !{path_to_demoparser_executable + " -dbname " + demo_name + " -dpath \"" + path_to_dem + "\""}

@client.on("error")
def handle_error(result):
    LOG.info("Logon result: %s", repr(result))
    
@client.on("chat_message")
def handle_chat_messame(user, message):
    LOG.info("Chat message from %s: %s", user.name, message)
    if message[0:6] != "steam:":
        user.send_message("Hi there, I'm not actually online. Pls contact me on my main acc (lolwto?!)")
        return
    else:
        user.send_message("Your match is being saved to the database, thank you for participating in the alpha!")
    link = message.replace('%20',' ').split()[-1]
    print(link)
    decoded = ''
    try:
        decoded = sharecode.decode(link)
    except ValueError:
        print("incorrect link")
        return
    print("decoded: ", decoded)
    goclient.decoded.put(sharecode.decode(link))
    goclient.launch()

@client.on("channel_secured")
def send_login():
    if client.relogin_available:
        client.relogin()

@client.on("connected")
def handle_connected():
    LOG.info("Connected to %s", client.current_server_addr)

@client.on("reconnect")
def handle_reconnect(delay):
    LOG.info("Reconnect in %ds...", delay)

@client.on("disconnected")
def handle_disconnect():
    LOG.info("Disconnected.")

    if client.relogin_available:
        LOG.info("Reconnecting...")
        client.reconnect(maxdelay=30)

@client.on("logged_on")
def handle_after_logon():
    LOG.info("-"*30)
    LOG.info("Logged on as: %s", client.user.name)
    LOG.info("Community profile: %s", client.steam_id.community_url)
    LOG.info("Last logon: %s", client.user.last_logon)
    LOG.info("Last logoff: %s", client.user.last_logoff)

In [5]:
def client_run():
    try:
        result = client.cli_login()

        if result != EResult.OK:
            LOG.info("Failed to login: %s" % repr(result))
            raise SystemExit

#         client.run_forever()
    except KeyboardInterrupt:
        if client.connected:
            LOG.info("Logout")
            client.logout()

In [6]:
# HTTPRequestHandler class
class testHTTPServer_RequestHandler(BaseHTTPRequestHandler):
 
  # GET
    def do_GET(self):
        self.send_response(200)
        self.send_header('Content-type','text/html')
        self.end_headers()
 
        message = "Hello world!"
        self.wfile.write(bytes(message, "utf8"))

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        body = self.rfile.read(content_length)
        fields = parse_qs(body)
        
        self.send_response(200)
        self.end_headers()
        response = BytesIO()
#         response.write(b'This is POST request. ')
        response.write(b'Received: ')
        response.write(body)
        self.wfile.write(response.getvalue())
        
        link = fields.get(b'param1', [b'no link'])[0].decode().split()[-1]
        print(link)
        decoded = ''        
        try:
            decoded = sharecode.decode(link)
        except ValueError:
            print("incorrect link")
            return
        print("decoded: ", decoded)
        goclient.decoded.put(decoded)
        goclient.launch()
#         goclient.wait_msg("ready", 5)
#         if goclient.connection_status == 0:
#             print("goclient.connection_status == 0")
#             return
#         goclient.request_full_match_info(**decoded)
        

In [7]:
def run():
    print('starting server...')
 
    server_address = ('127.0.0.1', 8081)
    httpd = HTTPServer(server_address, testHTTPServer_RequestHandler)
    print('running server...')
    httpd.serve_forever()
    
def find_values(json_repr, id='map'):
    results = []

    def _decode_dict(a_dict):
        try: results.append(a_dict[id])
        except KeyError: pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict)
    return results

In [8]:
client_run()

2019-06-16 22:59:50,949 | Connected to ('162.254.198.130', 27018)
2019-06-16 22:59:51,592 | ------------------------------
C:\Program Files\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
run()

starting server...
running server...


2019-06-16 22:59:53,455 | Logged on as: I've seen your elbow
2019-06-16 22:59:53,457 | Community profile: https://steamcommunity.com/profiles/76561198345386458
2019-06-16 22:59:53,457 | Last logon: 2019-06-16 19:47:25
2019-06-16 22:59:53,459 | Last logoff: 2019-06-16 19:47:21
127.0.0.1 - - [16/Jun/2019 22:59:57] "POST / HTTP/1.1" 200 -


CSGO-6bpsu-cqhc7-V7ffB-8yyKq-u2dOQ
decoded:  {'matchid': 3350660032361398514, 'outcomeid': 3350666507024597334, 'token': 11055}


2019-06-16 22:59:58,291 | Unable to find key for 2


Connected to MongoDB!
Header: map[ClientName:GOTV Demo Filestamp:HL2DEMO GameDirectory:csgo MapName:de_overpass NetworkProtocol:13700 PlaybackFrames:96391 PlaybackTicks:193071 PlaybackTime:50m16.734277632s Protocol:4 ServerName:Valve CS:GO Poland Server (srcds028.190.53) SignonLength:547761]
Connection to MongoDB closed.


...

Connected to MongoDB!
Header: map[ClientName:GOTV Demo Filestamp:HL2DEMO GameDirectory:csgo MapName:de_overpass NetworkProtocol:13700 PlaybackFrames:96391 PlaybackTicks:193071 PlaybackTime:50m16.734277632s Protocol:4 ServerName:Valve CS:GO Poland Server (srcds028.190.53) SignonLength:547761]
Connection to MongoDB closed.


2019-06-16 23:03:36,968 | Chat message from aydin: steam://rungame/730/76561202255233023/+csgo_download_match%20CSGO-6bpsu-cqhc7-V7ffB-8yyKq-u2dOQ


CSGO-6bpsu-cqhc7-V7ffB-8yyKq-u2dOQ
decoded:  {'matchid': 3350660032361398514, 'outcomeid': 3350666507024597334, 'token': 11055}


2019-06-16 23:03:37,610 | Unable to find key for 2


Connected to MongoDB!
Header: map[ClientName:GOTV Demo Filestamp:HL2DEMO GameDirectory:csgo MapName:de_overpass NetworkProtocol:13700 PlaybackFrames:96391 PlaybackTicks:193071 PlaybackTime:50m16.734277632s Protocol:4 ServerName:Valve CS:GO Poland Server (srcds028.190.53) SignonLength:547761]
Connection to MongoDB closed.
